<a href="https://colab.research.google.com/github/tnfru/abstraction_and_reasoning/blob/master/ARC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sys
if 'google.colab' in sys.modules:
  %tensorflow_version 2.x

  !wget https://raw.githubusercontent.com/tnfru/abstraction_and_reasoning/master/data_utils.py -P local_modules -nc
  sys.path.append('local_modules')

  from google_drive_downloader import GoogleDriveDownloader as gdd

In [7]:
import os, json
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Input
from tensorflow.keras.activations import softmax
from tensorflow.keras import Sequential

import data_utils
from data_utils import plot_task, plot_prediction, inp2img
import cellular_automata

### Set strategy

In [8]:
gpus = tf.config.experimental.list_logical_devices("GPU")

if len(gpus) > 0:
  strategy = tf.distribute.MirroredStrategy(gpus)
  print('Running on ', len(gpus), ' GPU(s) ')
else:
  strategy = tf.distribute.get_strategy()
  print('Running on CPU')

Running on CPU


## Read Data

In [9]:
if 'google.colab' in sys.modules:
  gdd.download_file_from_google_drive(file_id='1FUk30rXuym1RygfaDq4g-Uj-CkkG63IE',
                                      dest_path='./data/arc.zip',
                                      unzip=True)
train_tasks, val_tasks, test_tasks = data_utils.get_task_sets()

In [10]:
print("Number of equal input and output shapes in train:", data_utils.count_equal_dims(train_tasks), "of", len(train_tasks))
print("Number of equal input and output shapes in val:", data_utils.count_equal_dims(val_tasks), "of", len(val_tasks))
print("Number of equal input and output shapes in test:", data_utils.count_equal_dims(test_tasks, isTrain=False), "of", len(test_tasks))

Number of equal input and output shapes in train: 262 of 400
Number of equal input and output shapes in val: 270 of 400
Number of equal input and output shapes in test: 65 of 100


# Cellular Automata

For every problem where the input shape is equal to the output shape we can use a Cellular Automata for the transformation.

I want to thank [arseny-n](kaggle.com/arsenynerinovsky/cellular-automata-as-a-language-for-reasoning ) for introducing cellular automata to this problem set.

This notebook is an implementation of [teddykokers PyTorch CA](https://www.kaggle.com/teddykoker/training-cellular-automata-part-i-game-of-life) in tensorflow. I really like the the architecture teddy suggested, so I took the opportunity to practise translating a model from torch to tf.


## Vizualize results

In [6]:
cellular_automata.benchmark(train_tasks)

NameError: name 'cellular_automata' is not defined